3/8/25: Updated from the last stable/functional version ("dim1_BOW_train-test-combined").  Two major additions:  A different cross-validation method that does not require using **negative** mean absolute error.  I have also updated and expanded the "newsroom" test dataset in the hope of resolving the negative R2 issue.

7/14/25: implements the following changes from HK: BoW for transforming

In [ ]:
##import and format data
import pandas as pd
import numpy as np

#Upload complete training dataset
all114 = pd.read_csv('training_data_114_final.csv',dtype='string')
all2 = all114.astype({'speech':'string','nominate_dim1':'float', 'nominate_dim2': 'float'})
final114 = all2.dropna()


In [ ]:
#upload the custom stopword list
from congress_stopwords import congress


In [ ]:
final114.speech.str.len()
#Just making sure that no speech data were truncated during preprocessing
#and that the proper number of documents are present.

,speech
0,17784
1,25356
2,113985
3,109704
4,45340
...,...
433,90749
434,31889
435,15301
436,24527


NOTE: I TRIED THE VECTORIZER WITH AND WITHOUT LEMMATIZING.

In [ ]:
###lemmatize prior to vectorizer
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

W = []
lemmatizer = WordNetLemmatizer()
for i in final114.speech:
  lems = lemmatizer.lemmatize(i)
  W.append(lems)



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


BAG OF WORDS PREPROCESSING

In [ ]:
####create the BoW vectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
       stop_words=congress
    )
#train2 = vectorizer.fit_transform(W)
train1 = vectorizer.fit_transform(final114.speech)

In [ ]:
X = train1
###variable of NOMINATE values
y = final114.nominate_dim1
y1 = final114.nominate_dim2

In [ ]:
from sklearn.model_selection import RepeatedKFold, cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, max_error


In [ ]:
#TRAINING AND CROSS-VALIDATION -- DIMENSION 1
models = {
    'Linear Regression': LinearRegression(),
    #'Random Forest Regressor': RandomForestRegressor(n_estimators=600),
    #'Gradient Boosting Regressor': GradientBoostingRegressor(n_estimators=1600)
}

# Create a RepeatedKFold cross-validator
rkf = RepeatedKFold(n_splits=4, n_repeats=1, random_state=42)

# Function to calculate all metrics
def calculate_metrics(model, X, y, cv):
    r2_scores = []
    rmse_scores = []
    mae_scores = []
    max_errors = []

    for train_index, test_index in cv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calculate metrics
        r2_scores.append(r2_score(y_test, y_pred))
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        max_errors.append(max_error(y_test, y_pred))

    return r2_scores, rmse_scores, mae_scores, max_errors

# Evaluate each model using cross-validation and calculate the metrics
for model_name, model in models.items():
    r2_scores, rmse_scores, mae_scores, max_errors = calculate_metrics(model, X, y, rkf)

    #print(f"{model_name} - R2 Scores: {r2_scores}")
    print(f"{model_name} - Mean R2: {np.mean(r2_scores)}")
    #print(f"{model_name} - RMSE Scores: {rmse_scores}")
    print(f"{model_name} - Mean RMSE: {np.mean(rmse_scores)}")
    #print(f"{model_name} - MAE Scores: {mae_scores}")
    print(f"{model_name} - Mean MAE: {np.mean(mae_scores)}")
    #print(f"{model_name} - Max Errors: {max_errors}")
    print(f"{model_name} - Mean Max Error: {np.mean(max_errors)}\n")


Linear Regression - Mean R2: -2.228910147036488
Linear Regression - Mean RMSE: 0.8008730164016948
Linear Regression - Mean MAE: 0.5534039123848676
Linear Regression - Mean Max Error: 3.943388847785059

